# LSTM for Sentence Classification with PyTorch
[![Portfolio](https://img.shields.io/badge/Portfolio-chriskhanhtran.github.io-blue?logo=GitHub)](https://chriskhanhtran.github.io/)

## Introduction

Long Short-Term Memory (LSTM) neural networks are a special type of recurrent neural network (RNN) designed to model sequential data and capture long-range dependencies. Unlike traditional RNNs, which struggle with learning long-term patterns due to the vanishing gradient problem, LSTMs use a sophisticated memory cell and gating mechanism to retain and control information over extended sequences. This makes them especially effective for tasks such as natural language processing, time series forecasting, and speech recognition, where understanding the context and order of data is essential.


**Reference:**
- [Understanding LSTM--a tutorial into long short-term memory recurrent neural networks](https://arxiv.org/pdf/1909.09586)
- [Long Short-Term Memory](https://interactiveaudiolab.github.io/teaching/casa/HorchreiterSchmidhuber_LSTM.pdf)



## 1. Set up

### 1.1. Import Libraries

In [1]:
import os
import re
from tqdm import tqdm
import numpy as np
import pandas as pd
import nltk
nltk.download("all")
import matplotlib.pyplot as plt
import torch

# The following command allows matplotlib plots to appear inline with the notebook
# Rather than opening in another window, when you do not run any code cell
# you will get your plot below the code well.
%matplotlib inline

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_rus to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |  

### 1.2. Download Datasets

The dataset we will use is Movie Review (MR), a sentence polarity dataset from (Pang and Lee, 2005). The dataset has 5331 positive and 5331 negative processed sentences/snippets.

In [2]:
URL = 'https://www.cs.cornell.edu/people/pabo/movie-review-data/rt-polaritydata.tar.gz'
# Download Datasets
!wget -P 'Data/' $URL
# Unzip
!tar xvzf 'Data/rt-polaritydata.tar.gz' -C 'Data/'

--2025-04-07 15:35:20--  https://www.cs.cornell.edu/people/pabo/movie-review-data/rt-polaritydata.tar.gz
Resolving www.cs.cornell.edu (www.cs.cornell.edu)... 132.236.207.53
Connecting to www.cs.cornell.edu (www.cs.cornell.edu)|132.236.207.53|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 487770 (476K) [application/x-gzip]
Saving to: ‘Data/rt-polaritydata.tar.gz’

rt-polaritydata.tar 100%[===================>] 476.34K  1.38MB/s    in 0.3s    

2025-04-07 15:35:21 (1.38 MB/s) - ‘Data/rt-polaritydata.tar.gz’ saved [487770/487770]

rt-polaritydata.README.1.0.txt
rt-polaritydata/rt-polarity.neg
rt-polaritydata/rt-polarity.pos


In [3]:
def load_text(path):
    """Load text data, lowercase text and save to a list."""

    with open(path, 'rb') as f:
        texts = []
        for line in f:
            texts.append(line.decode(errors='ignore').lower().strip())

    return texts

# Load files
neg_text = load_text('Data/rt-polaritydata/rt-polarity.neg')
pos_text = load_text('Data/rt-polaritydata/rt-polarity.pos')

# Concatenate and label data
texts = np.array(neg_text + pos_text)
labels = np.array([0]*len(neg_text) + [1]*len(pos_text))

In [4]:
pos_text[0]

'the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .'

### 1.3. Download fastText Word Vectors

The pretrained word vectors used in the original paper is *word2vec* (Mikolov et al., 2013) trained on 100 billion tokens of Google News. In this tutorial, we will use [*fastText* pretrained word vectors](https://fasttext.cc/docs/en/english-vectors.html) (Mikolov et al., 2017), trained on 600 billion tokens on Common Crawl. *fastText* is an upgraded version of *word2vec* and outperform other state-of-the-art methods by a large margin.

The code below will download fastText pretrained vectors. Using Google Colab, the running time is approximately 3min 30s.

In [5]:
%%time
URL = "https://dl.fbaipublicfiles.com/fasttext/vectors-english/crawl-300d-2M.vec.zip"
FILE = "fastText"

if os.path.isdir(FILE):
    print("fastText exists.")
else:
    !wget -P $FILE $URL
    !unzip $FILE/crawl-300d-2M.vec.zip -d $FILE

--2025-04-07 15:35:25--  https://dl.fbaipublicfiles.com/fasttext/vectors-english/crawl-300d-2M.vec.zip
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 3.165.160.120, 3.165.160.69, 3.165.160.106, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|3.165.160.120|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1523785255 (1.4G) [application/zip]
Saving to: ‘fastText/crawl-300d-2M.vec.zip’

crawl-300d-2M.vec.z 100%[===================>]   1.42G   189MB/s    in 11s     

2025-04-07 15:35:36 (132 MB/s) - ‘fastText/crawl-300d-2M.vec.zip’ saved [1523785255/1523785255]

Archive:  fastText/crawl-300d-2M.vec.zip
  inflating: fastText/crawl-300d-2M.vec  
CPU times: user 368 ms, sys: 67.9 ms, total: 436 ms
Wall time: 1min 4s


### 1.4. Set up GPU for Training

Google Colab offers free GPUs and TPUs. Since we'll be training a large neural network it's best to utilize these features.

A GPU can be added by going to the menu and selecting:

> Runtime -> Change runtime type -> Hardware accelerator: GPU

Then we need to run the following cell to specify the GPU as the device.

In [6]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: Tesla T4


## 2. Data Preparation

To prepare our text data for training, first we need to tokenize our sentences and build a vocabulary dictionary `word2idx`, which will later be used to convert our tokens into indexes and build an embedding layer.

***So, what is an embedding layer?***

An embedding layer serves as a look-up table which take word indexes in the vocabulary as input and output word vectors. Hence, the embedding layer has shape $(N, d)$ where $N$ is the size of the vocabulary and $d$ is the embedding dimension. In order to fine-tune pretrained word vectors, we need to create an embedding layer in our `nn.Modules` class. Our input to the model will then be `input_ids`, which is the tokens' index in the vocabulary.

### 2.1. Tokenize

The function `tokenize` will tokenize our sentences, build a vocabulary and fine the maximum sentence length. The function `encode` will take in the outputs of `tokenize`, perform sentence padding and return `input_ids` as a numpy array.

In [7]:
from nltk.tokenize import word_tokenize
from collections import defaultdict

def tokenize(texts):
    """Tokenize texts, build vocabulary and find maximum sentence length.
    Args:
        texts (List[str]): List of text data
    Returns:
        tokenized_texts (List[List[str]]): List of list of tokens
        word2idx (Dict): Vocabulary built from the corpus
        max_len (int): Maximum sentence length
    """

    max_len = 0
    tokenized_texts = []
    word2idx = {}

    # Add <pad> and <unk> tokens to the vocabulary
    word2idx['<pad>'] = 0
    word2idx['<unk>'] = 1

    # Building our vocab from the corpus starting from index 2
    idx = 2
    for sent in texts:
        tokenized_sent = word_tokenize(sent)

        # Add `tokenized_sent` to `tokenized_texts`
        tokenized_texts.append(tokenized_sent)

        # Add new token to `word2idx`
        for token in tokenized_sent:
            if token not in word2idx:
                word2idx[token] = idx
                idx += 1

        # Update `max_len`
        max_len = max(max_len, len(tokenized_sent))
    return tokenized_texts, word2idx, max_len

def encode(tokenized_texts, word2idx, max_len):
    """Pad each sentence to the maximum sentence length and encode tokens to
    their index in the vocabulary.

    Returns:
        input_ids (np.array): Array of token indexes in the vocabulary with
            shape (N, max_len). It will the input of our CNN model.
    """

    input_ids = []
    for tokenized_sent in tokenized_texts:
        # Pad sentences to max_len
        tokenized_sent += ['<pad>'] * (max_len - len(tokenized_sent))

        # Encode tokens to input_ids
        input_id = [word2idx.get(token) for token in tokenized_sent]
        input_ids.append(input_id)

    return np.array(input_ids)

### 2.2. Load Pretrained Vectors

We will load the pretrain vectors for each tokens in our vocabulary. For tokens with no pretraiend vectors, we will initialize random word vectors with the same length and variance.

In [8]:
from tqdm import tqdm_notebook

def load_pretrained_vectors(word2idx, fname):
    """Load pretrained vectors and create embedding layers.
    Args:
        word2idx (Dict): Vocabulary built from the corpus
        fname (str): Path to pretrained vector file

    Returns:
        embeddings (np.array): Embedding matrix with shape (N, d) where N is
            the size of word2idx and d is embedding dimension
    """

    print("Loading pretrained vectors...")
    fin = open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')

    # fin.readlines() reads a single line from the file object fin
    # .split() splits the line into a list of strings, using whitespace as the delimiter
    # map(int, ...) converts each string in the list to an integer
    # The first row contains dimension information
    n, d = map(int, fin.readline().split())

    # Initilize random embeddings
    embeddings = np.random.uniform(-0.25, 0.25, (len(word2idx), d))
    embeddings[word2idx['<pad>']] = np.zeros((d,))

    # Load pretrained vectors
    count = 0
    for line in tqdm_notebook(fin):
        tokens = line.rstrip().split(' ')
        # The element on the first position is the token/word
        word = tokens[0]
        if word in word2idx:
            count += 1
            # The embeddings start from the second token
            embeddings[word2idx[word]] = np.array(tokens[1:], dtype=np.float32)

    print(f"There are {count} / {len(word2idx)} pretrained vectors found.")
    return embeddings

Now let's put above steps together.

In [9]:
# Tokenize, build vocabulary, encode tokens
print("Tokenizing...\n")
tokenized_texts, word2idx, max_len = tokenize(texts)
input_ids = encode(tokenized_texts, word2idx, max_len)

# Load pretrained vectors
embeddings = load_pretrained_vectors(word2idx, "fastText/crawl-300d-2M.vec")
embeddings = torch.tensor(embeddings)

Tokenizing...

Loading pretrained vectors...


<ipython-input-8-3d2e524c8433>:29: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for line in tqdm_notebook(fin):


0it [00:00, ?it/s]

There are 18522 / 20280 pretrained vectors found.


### 2.3. Create PyTorch DataLoader

We will create an iterator for our dataset using the torch DataLoader class. This will help save on memory during training and boost the training speed. The batch_size used in the paper is 50.

In [10]:
from torch.utils.data import (TensorDataset, DataLoader, RandomSampler,
                              SequentialSampler)

def data_loader(train_inputs, val_inputs, train_labels, val_labels,
                batch_size=50):
    """Convert train and validation sets to torch.Tensors and load them to
    DataLoader.
    """

    # Convert data type to torch.Tensor
    train_inputs, val_inputs, train_labels, val_labels =\
    tuple(torch.tensor(data) for data in
          [train_inputs, val_inputs, train_labels, val_labels])

    # Specify batch_size
    batch_size = 50

    # Create DataLoader for training data
    # TensorData is created by receiving one or more tensors to its constructor
    # These tensors should have the same first dimension, which represents the number of samples.
    # The TensorDataset directly stores the provided tensors, making it easy to access them for training or evaluation
    train_data = TensorDataset(train_inputs, train_labels)
    train_sampler = RandomSampler(train_data)
    # DataLoader wraps a dataset object, which provides access to your data
    # It creates an iterable that provides batches of data.
    # This is essentially because training deep learning models often involves processing data in smaller, manageable chunks, or batches
    # rather than the entire dataset at once.
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

    # Create DataLoader for validation data
    val_data = TensorDataset(val_inputs, val_labels)
    val_sampler = SequentialSampler(val_data)
    val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=batch_size)

    return train_dataloader, val_dataloader

We will use 90% of the dataset for training and 10% for validation.

In [11]:
from sklearn.model_selection import train_test_split

# Train Test Split
train_inputs, val_inputs, train_labels, val_labels = train_test_split(
    input_ids, labels, test_size=0.1, random_state=42)

# Load data to PyTorch DataLoader
train_dataloader, val_dataloader = \
data_loader(train_inputs, val_inputs, train_labels, val_labels, batch_size=50)

In [12]:
torch.__version__

'2.6.0+cu124'

## 3. Model

### 3.1. Create LSTM Model

In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class LSTM_NLP(nn.Module):
    def __init__(self,
                 hidden_dim=None,
                 output_dim=None,
                 n_layers=None,
                 bidirectional=False,
                 lstm_dropout=None,
                 pretrained_embedding=None,
                 freeze_embedding=False,
                 vocab_size=None,
                 embed_dim=300,
                 fc_dropout=0.5):
        super(LSTM_NLP, self).__init__()
        # Embedding layer
        if pretrained_embedding is not None:
            self.vocab_size, self.embed_dim = pretrained_embedding.shape
            # pretrained_embedding is a tensor containing the pre-trained word embeddings
            # it typically is loaded from a file or another source.
            # freeze is a boolean parameter that determines whether the embeddings are frozen or trainable
            # When freeze=False, the embedding vectors can be fine-tuned to better suit the specific task.
            self.embedding = nn.Embedding.from_pretrained(pretrained_embedding,
                                                          freeze=freeze_embedding)
        else:
            self.embed_dim = embed_dim
            # padding_idx, when specified, the embedding for this index will be zeros.
            # This is useful for padding sequences to the same length.
            self.embedding = nn.Embedding(num_embeddings=vocab_size,
                                          embedding_dim=self.embed_dim,
                                          padding_idx=0,
                                          max_norm=5.0)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, num_layers=n_layers, bidirectional=bidirectional, dropout=lstm_dropout, batch_first=True)
        self.fc = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        self.dropout = nn.Dropout(fc_dropout)


    def forward(self, input_ids):
        ## Input_ids contains multiple input lists, each of which corresponds to a data sample
        ## The embedding layer takes an input_id as the input, and produces an embedding matrix for the sentence
        x_embed = self.embedding(input_ids).float()
        ## Change the data type to float32 so that x_embd is acceptable to the lstm layer
        x_embed = x_embed.to(torch.float32)
        ## The LSTM layer produces and output and updates the hidden state and the cell memory
        output, (hidden, cell) = self.lstm(x_embed)
        ## Depending on whether bi-LSTM is used, the final hidden state will take a different form
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1) if self.lstm.bidirectional else hidden[-1,:,:])
        ## Apply a dropout layer to prevent overfitting and a fully connected layer before a prediction is made
        return self.fc(self.dropout(hidden))

### 3.2. Optimizer

To train Deep Learning models, we need to define a loss function and minimize this loss. We'll use back-propagation to compute gradients and use an optimization algorithm (ie. Gradient Descent) to minimize the loss. The original paper used the Adadelta optimizer.

In [14]:
import torch.optim as optim
def initialize_lstm(pretrained_embedding=None,
                    freeze_embedding=False,
                    vocab_size=None,
                    embed_dim=300,
                    hidden_dim=256,
                    output_dim=2,
                    n_layers=2,
                    bidirectional=False,
                    lstm_dropout=0.5,
                    fc_dropout=0.5,
                    learning_rate=0.01):
    """Instantiate an LSTM model and an optimizer."""

    # Instantiate LSTM model
    lstm_model = LSTM_NLP(pretrained_embedding=pretrained_embedding,
                        freeze_embedding=freeze_embedding,
                        vocab_size=vocab_size,
                        embed_dim=embed_dim,
                        hidden_dim=hidden_dim,
                        output_dim=output_dim,
                        n_layers=n_layers,
                        bidirectional=bidirectional,
                        lstm_dropout=lstm_dropout,
                        fc_dropout=fc_dropout)

    # Send model to `device` (GPU/CPU)
    lstm_model.to(device)

    # Instantiate Adadelta optimizer
    optimizer = optim.Adadelta(lstm_model.parameters(),
                               lr=learning_rate,
                               rho=0.95)

    return lstm_model, optimizer

### 3.3. Training Loop

For each epoch, the code below will perform a forward step to compute the *Cross Entropy* loss, a backward step to compute gradients and use the optimizer to update weights/parameters. At the end of each epoch, the loss on training data and the accuracy over the validation data will be printed to help us keep track of the model's performance. The code is heavily annotated with detailed explanations.

In [15]:
import random
import time
loss_fn = nn.CrossEntropyLoss()

def set_seed(seed_value=42):
    """Set seed for reproducibility."""
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)

def train_lstm(model, optimizer, train_dataloader, val_dataloader=None, epochs=10):
    """Train the LSTM model."""
    # Tracking best validation accuracy
    best_accuracy = 0

    # Start training loop
    print("Start training...\n")
    print(f"{'Epoch':^7} | {'Train Loss':^12} | {'Val Loss':^10} | {'Val Acc':^9} | {'Elapsed':^9}")
    print("-"*60)

    for epoch_i in range(epochs):
        # =======================================
        #               Training
        # =======================================
        t0_epoch = time.time()
        total_loss = 0
        model.train()

        for step, batch in enumerate(train_dataloader):
            b_input_ids, b_labels = tuple(t.to(device) for t in batch)
            model.zero_grad()
            logits = model(b_input_ids)
            loss = loss_fn(logits, b_labels)
            total_loss += loss.item()
            loss.backward()
            optimizer.step()

        avg_train_loss = total_loss / len(train_dataloader)
        print(epoch_i, avg_train_loss)
        # =======================================
        #               Evaluation
        # =======================================
        if val_dataloader is not None:
            val_loss, val_accuracy = evaluate(model, val_dataloader)

            if val_accuracy > best_accuracy:
                best_accuracy = val_accuracy

            time_elapsed = time.time() - t0_epoch
            print(f"{epoch_i + 1:^7} | {avg_train_loss:^12.6f} | {val_loss:^10.6f} | {val_accuracy:^9.2f} | {time_elapsed:^9.2f}")
    print("\n")
    print(f"Training complete! Best accuracy: {best_accuracy:.2f}%.")

def evaluate(model, val_dataloader):
    """After the completion of each training epoch, measure the model's
    performance on our validation set.
    """
    # Put the model into the evaluation mode. The dropout layers are disabled
    # during the test time.
    model.eval()

    # Tracking variables
    val_accuracy = []
    val_loss = []

    # For each batch in our validation set...
    for batch in val_dataloader:
        # Load batch to GPU
        b_input_ids, b_labels = tuple(t.to(device) for t in batch)

        # Compute logits
        # During inference or evaluation, you don't need to compute gradients,
        # Hence, torch.no_grad is used to disable gradient calculation.
        with torch.no_grad():
            logits = model(b_input_ids)

        # Compute loss
        loss = loss_fn(logits, b_labels)
        val_loss.append(loss.item())

        # Get the predictions
        preds = torch.argmax(logits, dim=1).flatten()

        # Calculate the accuracy rate
        accuracy = (preds == b_labels).cpu().numpy().mean()*100
        val_accuracy.append(accuracy)

    # Compute the average accuracy and loss over the validation set.
    val_loss = np.mean(val_loss)
    val_accuracy = np.mean(val_accuracy)

    return val_loss, val_accuracy

In [20]:
# The following LSTM model does not use pre-trained word embedding
# Instead, its embedding layer is randomly initalized and is trainable in the training process
# Besides, the model is uni-directional and has 2 LSTM layers.

lstm_model, optimizer = initialize_lstm(pretrained_embedding=None,
                                        freeze_embedding=False,
                                        vocab_size=len(word2idx),
                                        embed_dim=300,
                                        hidden_dim=128,  # Example hidden dimension
                                        output_dim=2,  # Number of classes
                                        n_layers=2,
                                        bidirectional=False,  # Set to True if using bidirectional LSTM
                                        learning_rate = 0.25,
                                        lstm_dropout=0.2,
                                        fc_dropout=0.5)


In [21]:
train_lstm(lstm_model, optimizer, train_dataloader, val_dataloader, epochs=10)

Start training...

 Epoch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elapsed 
------------------------------------------------------------
0 0.6926077685008446
   1    |   0.692608   |  0.688831  |   54.51   |   3.19   
1 0.678282112814486
   2    |   0.678282   |  0.642891  |   62.58   |   3.15   
2 0.5788077563047409
   3    |   0.578808   |  0.522598  |   72.39   |   3.20   
3 0.5145847893630465
   4    |   0.514585   |  0.478816  |   75.94   |   3.11   
4 0.47651735118900734
   5    |   0.476517   |  0.496839  |   75.66   |   3.12   
5 0.44495242026944953
   6    |   0.444952   |  0.506546  |   76.66   |   3.12   
6 0.4278572377904008
   7    |   0.427857   |  0.441814  |   80.93   |   3.20   
7 0.3975615656624238
   8    |   0.397562   |  0.451639  |   79.74   |   3.20   
8 0.37902200742003817
   9    |   0.379022   |  0.432811  |   81.83   |   3.13   
9 0.35968827184600133
  10    |   0.359688   |  0.439961  |   81.01   |   3.14   


Training complete! Best accuracy: 81.83%.
